# itch_wang_res_comparison

#### Juan Camilo Henao Londono - 07.03.2019
#### AG Guhr - Universitaet Duisburg-Essen

## Step 1

Obtain the data in the same format of S. Wang for the stock AAPL 2008.01.07. It is used the algorithm I implemented.

In [1]:
import gzip
import numpy as np
import pandas as pd

import itch_taq_trade_sign_classification_test_basic as itch_sign_clas

In [2]:
ticker = 'AAPL'
year = '2008'
month = '06'
day = '02'
filename_transaction = 'SignCompare_transactions_20080602_AAPL.txt'
filename_persecond = 'SignCompare_persecond_20080602_AAPL.txt'
filename_transaction_corr = 'SignCompare_transactions_20080602_AAPL_corr.txt'
filename_persecond_corr = 'SignCompare_persecond_20080602_AAPL_corr.txt'

In [3]:
times_signs, trade_signs, vol_signs, price_signs = itch_sign_clas.itch_taq_trade_signs_load_test(ticker, year,
                                                                                                 month, day)
id_trades = itch_sign_clas.itch_taq_trade_signs_eq1_ms_test(ticker, trade_signs, price_signs, year, month, day)
teo_s, exp_eq2_s, times_s = itch_sign_clas.itch_taq_trade_signs_eq2_s_test(ticker, times_signs, trade_signs,
                                                                           id_trades, year, month, day)
_, exp_eq3_s, _ = itch_sign_clas.itch_taq_trade_signs_eq3_s_test(ticker, times_signs, trade_signs,
                                                                 vol_signs, id_trades, year, month, day)

Accuracy of the trade sign classification for consecutive trades in ms for the stock AAPL the 2008.06.02
Accuracy of the trade sign classification for number imbalance of  trades in seconds for the stock AAPL the 2008.06.02
Accuracy of the trade sign classification for volume imbalance of  trades in ms for the stock AAPL the 2008.06.02


In [4]:
trades_no_0 = trade_signs != 0
d1 = {'Time': times_signs[trades_no_0], 'Teo': trade_signs[trades_no_0],
      'Exp': id_trades, 'Price': price_signs[trades_no_0]}
transactions_juan = pd.DataFrame(data=d1).astype(int)
transactions_juan['Price'] = transactions_juan['Price'] / 10000

In [5]:
d2 = {'Time': times_s * 1000, 'Eq2': exp_eq2_s, 'Eq3': exp_eq3_s, 'Teo': teo_s}
persecond_juan = pd.DataFrame(data=d2).astype(int)

## Step 2

Load S. Wang data.

In [6]:
transactions_wang = pd.read_csv(filename_transaction, sep='   ',
                                usecols=(1,3,4,5), header=None, engine='python')
transactions_wang.columns = ['Time', 'Teo', 'Exp', 'Price']

In [7]:
persecond_wang = pd.read_csv(filename_persecond, sep='   ',
                             usecols=(0,2,3,4), header=None, engine='python')
persecond_wang.columns = ['Time', 'Eq2', 'Eq3', 'Teo']

## Step 3

Compare the data to see where are the difference

In [8]:
trans_time_comp = np.sum(transactions_wang['Time'] == transactions_juan['Time']) / len(transactions_wang['Time'])
perse_time_comp = np.sum(persecond_wang['Time'] == persecond_juan['Time']) / len(persecond_wang['Time'])

print('The similarity of the time in the transaction time used is {:.2f}%'.format(trans_time_comp * 100))
print('The similarity of the time in the persecond time used is   {:.2f}%'.format(perse_time_comp * 100))

The similarity of the time in the transaction time used is 100.00%
The similarity of the time in the persecond time used is   100.00%


In [9]:
trans_teo_comp = np.sum(transactions_wang['Teo'] == transactions_juan['Teo']) / len(transactions_wang['Teo'])
perse_teo_comp = np.sum(persecond_wang['Teo'] == persecond_juan['Teo']) / len(persecond_wang['Teo'])

print('The similarity of the reference trade signs values for the transactions is {:.2f}%'.format(trans_teo_comp * 100))
print('The similarity of the reference trade signs values for the persecond is    {:.2f}%'.format(perse_teo_comp * 100))

The similarity of the reference trade signs values for the transactions is 100.00%
The similarity of the reference trade signs values for the persecond is    93.05%


In [10]:
trans_exp_comp = np.sum(transactions_wang['Exp'] == transactions_juan['Exp']) / len(transactions_wang['Exp'])
perse_eq2_comp = np.sum(persecond_wang['Eq2'] == persecond_juan['Eq2']) / len(persecond_wang['Eq2'])
perse_eq3_comp = np.sum(persecond_wang['Eq3'] == persecond_juan['Eq3']) / len(persecond_wang['Eq3'])

print('The similarity of the experimental result for the transactions is {:.2f}%'.format(trans_exp_comp * 100))
print('The similarity of the Eq. 2 result for the persecond is           {:.2f}%'.format(perse_eq2_comp * 100))
print('The similarity of the Eq. 3 result for the persecond is           {:.2f}%'.format(perse_eq3_comp * 100)) 

The similarity of the experimental result for the transactions is 100.00%
The similarity of the Eq. 2 result for the persecond is           93.04%
The similarity of the Eq. 3 result for the persecond is           91.13%


In [11]:
eq_2_3_sim_juan = np.sum(persecond_juan['Eq2'] == persecond_juan['Eq3']) / len(persecond_juan['Eq2'])
print(eq_2_3_sim_juan)
eq_2_3_sim_wang = np.sum(persecond_wang['Eq2'] == persecond_wang['Eq3']) / len(persecond_wang['Eq2'])
print(eq_2_3_sim_wang)

0.9761711711711711
0.9717567567567568


## Step 4

Check the differences in every experimental result in the millisecond result and determine the error.

In [12]:
trans_exp_diff = np.where(transactions_wang['Exp'] != transactions_juan['Exp'])[0]

print('The total of different values between the experimental results are ', len(trans_exp_diff))
print('The first ten different values are located in the positions ', trans_exp_diff[:10])

The total of different values between the experimental results are  1
The first ten different values are located in the positions  [0]


In [13]:
print('The first value of Wang is {} and of Juan is {}'
      .format(transactions_wang['Exp'][0], transactions_juan['Exp'][0]))

The first value of Wang is -1 and of Juan is 1


In [14]:
wang = 0
juan = 0

for val in trans_exp_diff:
    
    print('The value in pos. {} of Wang is {} and of Juan is {}'
          .format(val, transactions_wang['Exp'][val], transactions_juan['Exp'][val]))
    print('To check which value is correct, we need to find the value in the position {} and {}'
          .format(val, val - 1))
    sign = np.sign(price_signs[val] - price_signs[val - 1])
    print('The price in pos. {} is {} and in position {} is {}. Then the trade sign must be {}'
              .format(val, price_signs[val], val - 1, price_signs[val - 1], sign))
    if (sign == 0):
        print(' '.join(('As the sign is zero, we use the trade sign value in pos. {}. For Wang that value'
              + ' is {} and for Juan that value is {}. The real value of that position is {}').split())
              .format(val - 1, transactions_wang['Exp'][val - 1], transactions_juan['Exp'][val - 1],
                      trade_signs[val - 1]))
        if (transactions_wang['Exp'][val - 1] == trade_signs[val - 1]
            and transactions_juan['Exp'][val - 1] == trade_signs[val - 1]):
            print('Both were right')
            wang += 1
            juan += 1
        elif (transactions_wang['Exp'][val - 1] == trade_signs[val - 1]):
            print('Wang was right')
            wang += 1
        else:
            print('Juan was right')
            juan += 1
    else:
        print('The real value of that position is {}'.format(trade_signs[val]))
        if (trade_signs[val] == transactions_wang['Exp'][val]):
            print('Wang was right')
            wang += 1
        else:
            print('Juan was right')
            juan += 1
    print()

print('Wang was right {} times and Juan was right {} times'.format(wang, juan))    
print()

The value in pos. 0 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the position 0 and -1
The price in pos. 0 is 188.63 and in position -1 is 185.63. Then the trade sign must be 1.0
The real value of that position is -1.0
Wang was right

Wang was right 1 times and Juan was right 0 times



The reason of this error is due to the fact I just use the values from the open market time. Mi first value is calculated using the difference between the first and last value `diff = price_signs[0] - price_signs[-1]`, then my first value is error prone.

## Step 5

Check the differences in every experimental result in the Eq. 1 result and determine the error.

In [15]:
trans_eq2_diff = np.where(persecond_wang['Eq2'] != persecond_juan['Eq2'])[0]

print('The total of different values between the experimental results are ', len(trans_eq2_diff))
print('The first ten different values are located in the positions ', trans_eq2_diff[:10])

The total of different values between the experimental results are  1545
The first ten different values are located in the positions  [ 0 38 46 53 63 72 74 77 86 91]


In [16]:
wang = 0
juan = 0

for val in trans_eq2_diff:
    
    print('The value in pos. {} of Wang is {} and of Juan is {}'
          .format(val, persecond_wang['Eq2'][val], persecond_juan['Eq2'][val]))
    print('To check which value is correct, we need to find the value in the second {}'
          .format(34800 + val))
    
    condition = (transactions_wang['Time'] / 1000 >= 34800 + val) \
                * (transactions_wang['Time'] / 1000 < 34800 + val + 1)
    print('Juan: ', list(transactions_juan['Exp'][condition]))
    print('Wang: ', list(transactions_wang['Exp'][condition]))
    
    trades_sign_eq2_juan = np.sum(transactions_juan['Exp'][condition])
    trades_sign_eq2_wang = np.sum(transactions_wang['Exp'][condition])
    trades_sign_eq2_teo = np.sum(transactions_wang['Teo'][condition])
    
    print('The sum of Juan is {}, the sum of Wang is {}, the theoretical value is {}'
          .format(trades_sign_eq2_juan, trades_sign_eq2_wang, np.sign(trades_sign_eq2_teo)))                   

    if (np.sign(trades_sign_eq2_juan) == np.sign(trades_sign_eq2_teo)):
        print('Juan was right')
        juan += 1
    elif (np.sign(trades_sign_eq2_wang) == np.sign(trades_sign_eq2_teo)):
        print('Wang was right')
        wang += 1
    else:
        print('Hi')

    print()

print('Wang was right {} times and Juan was right {} times'.format(wang, juan))    
print()

The value in pos. 0 of Wang is -1 and of Juan is 0
To check which value is correct, we need to find the value in the second 34800
Juan:  [1, 1, -1, -1]
Wang:  [-1, 1, -1, -1]
The sum of Juan is 0, the sum of Wang is -2, the theoretical value is -1
Wang was right

The value in pos. 38 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 34838
Juan:  [-1, -1, -1, -1]
Wang:  [-1, -1, -1, -1]
The sum of Juan is -4, the sum of Wang is -4, the theoretical value is 1
Hi

The value in pos. 46 of Wang is 1 and of Juan is 0
To check which value is correct, we need to find the value in the second 34846
Juan:  [-1, 1]
Wang:  [-1, 1]
The sum of Juan is 0, the sum of Wang is 0, the theoretical value is 1
Hi

The value in pos. 53 of Wang is 0 and of Juan is 1
To check which value is correct, we need to find the value in the second 34853
Juan:  [-1, -1, 1, 1, 1, 1, 1, 1]
Wang:  [-1, -1, 1, 1, 1, 1, 1, 1]
The sum of Juan is 4, the sum of Wang is 4, the

Juan:  [-1]
Wang:  [-1]
The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 345 of Wang is 0 and of Juan is -1
To check which value is correct, we need to find the value in the second 35145
Juan:  [-1, -1, -1, -1, 1, 1]
Wang:  [-1, -1, -1, -1, 1, 1]
The sum of Juan is -2, the sum of Wang is -2, the theoretical value is 0
Hi

The value in pos. 360 of Wang is 0 and of Juan is 1
To check which value is correct, we need to find the value in the second 35160
Juan:  [1, 1]
Wang:  [1, 1]
The sum of Juan is 2, the sum of Wang is 2, the theoretical value is 0
Hi

The value in pos. 368 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 35168
Juan:  [-1]
Wang:  [-1]
The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 446 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 35246
Juan:  [-1, -1, -1, -1, 1, 1, 1

/home/tp/jchenaol/.pyenv/versions/3.6.5/lib/python3.6/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Juan:  [-1, -1]
Wang:  [-1, -1]
The sum of Juan is -2, the sum of Wang is -2, the theoretical value is 0
Hi

The value in pos. 482 of Wang is 0 and of Juan is 1
To check which value is correct, we need to find the value in the second 35282
Juan:  [1, 1, 1, 1]
Wang:  [1, 1, 1, 1]
The sum of Juan is 4, the sum of Wang is 4, the theoretical value is 0
Hi

The value in pos. 504 of Wang is 0 and of Juan is 1
To check which value is correct, we need to find the value in the second 35304
Juan:  [1, 1, 1, -1]
Wang:  [1, 1, 1, -1]
The sum of Juan is 2, the sum of Wang is 2, the theoretical value is 0
Hi

The value in pos. 520 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 35320
Juan:  [-1]
Wang:  [-1]
The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 524 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the second 35324
Juan:  [1]
Wang:  [1]
The sum of

The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 856 of Wang is 0 and of Juan is -1
To check which value is correct, we need to find the value in the second 35656
Juan:  [-1, -1]
Wang:  [-1, -1]
The sum of Juan is -2, the sum of Wang is -2, the theoretical value is 0
Hi

The value in pos. 891 of Wang is -1 and of Juan is 0
To check which value is correct, we need to find the value in the second 35691
Juan:  [1, 1, -1, -1]
Wang:  [1, 1, -1, -1]
The sum of Juan is 0, the sum of Wang is 0, the theoretical value is -1
Hi

The value in pos. 894 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 35694
Juan:  [-1, -1]
Wang:  [-1, -1]
The sum of Juan is -2, the sum of Wang is -2, the theoretical value is 1
Hi

The value in pos. 907 of Wang is 0 and of Juan is -1
To check which value is correct, we need to find the value in the second 35707
Juan:  [-1, -1, 1, 1, -1, -1]
Wang:  [-1, -1, 1, 1, -1, -

...

Juan:  [1, 1, -1, -1, -1, 1, 1]
Wang:  [1, 1, -1, -1, -1, 1, 1]
The sum of Juan is 1, the sum of Wang is 1, the theoretical value is -1
Hi

The value in pos. 20160 of Wang is -1 and of Juan is 0
To check which value is correct, we need to find the value in the second 54960
Juan:  [1, 1, -1, -1]
Wang:  [1, 1, -1, -1]
The sum of Juan is 0, the sum of Wang is 0, the theoretical value is -1
Hi

The value in pos. 20189 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 54989
Juan:  [-1]
Wang:  [-1]
The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 20217 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 55017
Juan:  [-1, -1]
Wang:  [-1, -1]
The sum of Juan is -2, the sum of Wang is -2, the theoretical value is 1
Hi

The value in pos. 20242 of Wang is 0 and of Juan is 1
To check which value is correct, we need to find the value in the second 5

The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 20644 of Wang is 0 and of Juan is -1
To check which value is correct, we need to find the value in the second 55444
Juan:  [-1, -1]
Wang:  [-1, -1]
The sum of Juan is -2, the sum of Wang is -2, the theoretical value is 0
Hi

The value in pos. 20655 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the second 55455
Juan:  [1, 1, 1]
Wang:  [1, 1, 1]
The sum of Juan is 3, the sum of Wang is 3, the theoretical value is -1
Hi

The value in pos. 20656 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the second 55456
Juan:  [1]
Wang:  [1]
The sum of Juan is 1, the sum of Wang is 1, the theoretical value is -1
Hi

The value in pos. 20677 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the second 55477
Juan:  [1, 1, 1]
Wang:  [1, 1, 1]
The sum of Juan is 3, the sum of Wan

Juan:  [1, 1, 1, 1, 1, -1]
Wang:  [1, 1, 1, 1, 1, -1]
The sum of Juan is 4, the sum of Wang is 4, the theoretical value is -1
Hi

The value in pos. 20992 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 55792
Juan:  [-1, -1, -1, -1, -1, -1, -1]
Wang:  [-1, -1, -1, -1, -1, -1, -1]
The sum of Juan is -7, the sum of Wang is -7, the theoretical value is 1
Hi

The value in pos. 20996 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the second 55796
Juan:  [1, 1, 1]
Wang:  [1, 1, 1]
The sum of Juan is 3, the sum of Wang is 3, the theoretical value is -1
Hi

The value in pos. 21027 of Wang is 0 and of Juan is -1
To check which value is correct, we need to find the value in the second 55827
Juan:  [-1, -1]
Wang:  [-1, -1]
The sum of Juan is -2, the sum of Wang is -2, the theoretical value is 0
Hi

The value in pos. 21035 of Wang is -1 and of Juan is 0
To check which value is correct, we need to f

To check which value is correct, we need to find the value in the second 56203
Juan:  [1]
Wang:  [1]
The sum of Juan is 1, the sum of Wang is 1, the theoretical value is -1
Hi

The value in pos. 21404 of Wang is 0 and of Juan is 1
To check which value is correct, we need to find the value in the second 56204
Juan:  [1, 1, 1, 1]
Wang:  [1, 1, 1, 1]
The sum of Juan is 4, the sum of Wang is 4, the theoretical value is 0
Hi

The value in pos. 21410 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 56210
Juan:  [-1, -1, 1]
Wang:  [-1, -1, 1]
The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 21428 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 56228
Juan:  [-1]
Wang:  [-1]
The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 21432 of Wang is 0 and of Juan is 1
To check which value is correct, we n

The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 21898 of Wang is 1 and of Juan is -1
To check which value is correct, we need to find the value in the second 56698
Juan:  [-1]
Wang:  [-1]
The sum of Juan is -1, the sum of Wang is -1, the theoretical value is 1
Hi

The value in pos. 21916 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the second 56716
Juan:  [1, 1, 1]
Wang:  [1, 1, 1]
The sum of Juan is 3, the sum of Wang is 3, the theoretical value is -1
Hi

The value in pos. 21917 of Wang is -1 and of Juan is 1
To check which value is correct, we need to find the value in the second 56717
Juan:  [1, 1, 1, 1, 1]
Wang:  [1, 1, 1, 1, 1]
The sum of Juan is 5, the sum of Wang is 5, the theoretical value is -1
Hi

The value in pos. 21918 of Wang is -1 and of Juan is 0
To check which value is correct, we need to find the value in the second 56718
Juan:  [1, -1, -1, -1, -1, 1, 1, 1]
Wang:  [1, -1, -1,

## Step 6

S. Wang confuse the output for the second accuracy result. I change her code and obtain the correct data. With that this are the new results of the comparison

In [17]:
# Load new data
transactions_wang_corr = pd.read_csv(filename_transaction_corr, sep='   ',
                                usecols=(1,3,4,5), header=None, engine='python')
transactions_wang_corr.columns = ['Time', 'Teo', 'Exp', 'Price']

persecond_wang_corr = pd.read_csv(filename_persecond_corr, sep='   ',
                             usecols=(0,2,3,4), header=None, engine='python')
persecond_wang_corr.columns = ['Time', 'Eq2', 'Eq3', 'Teo']

In [18]:
# Comparison

trans_time_comp = np.sum(transactions_wang_corr['Time'] == transactions_juan['Time']) / len(transactions_wang_corr['Time'])
perse_time_comp = np.sum(persecond_wang_corr['Time'] == persecond_juan['Time']) / len(persecond_wang_corr['Time'])

print('The similarity of the time in the transaction time used is {:.2f}%'.format(trans_time_comp * 100))
print('The similarity of the time in the persecond time used is   {:.2f}%'.format(perse_time_comp * 100))

trans_teo_comp = np.sum(transactions_wang_corr['Teo'] == transactions_juan['Teo']) / len(transactions_wang_corr['Teo'])
perse_teo_comp = np.sum(persecond_wang_corr['Teo'] == persecond_juan['Teo']) / len(persecond_wang_corr['Teo'])

print('The similarity of the reference trade signs values for the transactions is {:.2f}%'.format(trans_teo_comp * 100))
print('The similarity of the reference trade signs values for the persecond is    {:.2f}%'.format(perse_teo_comp * 100))

trans_exp_comp = np.sum(transactions_wang_corr['Exp'] == transactions_juan['Exp']) / len(transactions_wang_corr['Exp'])
perse_eq2_comp = np.sum(persecond_wang_corr['Eq2'] == persecond_juan['Eq2']) / len(persecond_wang_corr['Eq2'])
perse_eq3_comp = np.sum(persecond_wang_corr['Eq3'] == persecond_juan['Eq3']) / len(persecond_wang_corr['Eq3'])

print('The similarity of the experimental result for the transactions is {:.2f}%'.format(trans_exp_comp * 100))
print('The similarity of the Eq. 2 result for the persecond is           {:.2f}%'.format(perse_eq2_comp * 100))
print('The similarity of the Eq. 3 result for the persecond is           {:.2f}%'.format(perse_eq3_comp * 100)) 

The similarity of the time in the transaction time used is 100.00%
The similarity of the time in the persecond time used is   100.00%
The similarity of the reference trade signs values for the transactions is 100.00%
The similarity of the reference trade signs values for the persecond is    100.00%
The similarity of the experimental result for the transactions is 100.00%
The similarity of the Eq. 2 result for the persecond is           100.00%
The similarity of the Eq. 3 result for the persecond is           96.47%
